# Find the biggest possible watermelon of least regret problem

Here is an interesting problem. 

Having N watermelons with random size, how can you pick the biggest possible one given:
- You can only see one watermelon for each time and can only see it for once 
- Once you decice to pick one, you are done.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool

"""
Given N watermelons with random size, if you can only see a  
watermelon for once and decide whether to pick it, 
how to pick the biggest possible one if you can only pick for once?
"""
def find_biggest_least_regret(param):
    """ Have N watermelons, find the best window size and the percetage 
        that has lest regret """
    W, P, N = param
    num_trials=10000 
    total_regret = 0
    for i in range(num_trials):
        watermelons = np.random.rand(N)
        L = np.sort(watermelons[:W])[-int(W * P)]
        find_bigger = watermelons[W:][watermelons[W:] >= L]
        if find_bigger.shape[0] > 0:
            result = find_bigger[0]
        else:
            result = watermelons[-1]

        ## regret
        watermelons_sorted = np.sort(watermelons)
        rank = list(watermelons_sorted).index(result)
        regret = N - 1 - rank
        total_regret += regret
    return total_regret/ num_trials


def best_param_for_N(N):
    params = [[W, P, N] for P in np.linspace(0.01, 0.5, 11) for W in range(1, N)]
    num_cpus = 12
    with Pool(num_cpus) as p:
        average_regret = p.map(find_biggest_least_regret, params)

    min_index = np.argmin(average_regret)
    W, P, N = params[min_index]
    print(f'N: {N} sample window: {W/N*100}% decision window: {P*100}% average regret: {average_regret[min_index]}')

    
if __name__=='__main__':
    for N in [5, 10, 50, 100, 200, 1000]:
        best_param_for_N(N)


N: 5 sample window: 20.0% decision window: 1.0% average regret: 1.0802
N: 10 sample window: 40.0% decision window: 50.0% average regret: 1.9119
N: 50 sample window: 38.0% decision window: 20.6% average regret: 4.0458
N: 100 sample window: 34.0% decision window: 10.8% average regret: 4.9898
N: 200 sample window: 38.0% decision window: 5.9% average regret: 5.8873
N: 1000 sample window: 41.4% decision window: 1.0% average regret: 15.6848


Conclusion
Our strategy is
- Check and remember the sizes of the first W samples
- In the rest N-W samples, pick the first one that is no smaller than (1-P) percentage of the first W samples

Based on out simulation results, except for N=5, we find the best sample window is nearly constant, which is about 40% of N. The decision making percentage P decreases with N, which makes sense because the larger the total number N is, the more close the size districbution in the first W is to that of the rest (N-W) samples.    

